In [2]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

2.7.1+cu118
True
cuda:0


In [14]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import optuna
from torch.profiler import profile, ProfilerActivity
import time 
from torch.cuda.amp import GradScaler, autocast 

# Check if the environment variable is set to 'True'
skip_training = os.getenv('SKIP_TRAINING', 'False') == 'True'
skip_optuna = os.getenv('SKIP_OPTUNA', 'False') == 'True'

PROCESSED_DATA_DIR = '../dataset/LG_HG2_processed'
FEATURE_COLS = ['Voltage [V]', 'Current [A]', 'Temperature [degC]', 'Power [W]', 'CC_Capacity [Ah]']
LABEL_COL = 'SOC [-]'
BATCH_SIZE = 128
EPOCHS = 10
LEARNING_RATE = 0.0001
SEQUENCE_LENGTH = 20  
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Function to load data
def load_data(directory, temperatures):
    frames = []    
    for temp_folder in os.listdir(directory):
        if temp_folder in temperatures:
            temp_path = os.path.join(directory, temp_folder)
            for file in os.listdir(temp_path):
                if 'Charge' in file or 'Dis' in file:
                    continue  # Skip constant charge and discharge files
                if file.endswith('.csv'):
                    df = pd.read_csv(os.path.join(temp_path, file))
                    df['SourceFile'] = file

                    # Calculate power
                    df['Power [W]'] = df['Voltage [V]'] * df['Current [A]']

                    # Initialize CC_Capacity [Ah] column
                    df['CC_Capacity [Ah]'] = 0.0

                    # Integrating current over time to calculate cumulative capacity
                    df['CC_Capacity [Ah]'] = (df['Current [A]'] * df['Time [s]'].diff().fillna(0) / 3600).cumsum()

                    frames.append(df)
    return pd.concat(frames, ignore_index=True)


# Custom dataset class for LSTM
class BatteryDatasetLSTM(Dataset):
    def __init__(self, data_tensor, labels_tensor, sequence_length=50, filenames=None, times=None):
        self.sequence_length = sequence_length
        self.features = data_tensor
        self.labels = labels_tensor
        self.filenames = filenames 
        self.times = times 

    def __len__(self):
        return len(self.features) - self.sequence_length

    def __getitem__(self, idx):
        idx_end = idx + self.sequence_length
        sequence = self.features[idx:idx_end]
        label = self.labels[idx_end - 1]
        filename = self.filenames[idx_end - 1]
        time = self.times[idx_end - 1]  

        sequence = sequence.clone().detach()
        label = label.clone().detach()

        return sequence, label, filename, time
    
    def get_unique_filenames(self):
        return set(self.filenames)
    
    def get_times(self):
        return self.times

# SoCLSTM Model
class SoCLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(SoCLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size, dtype=x.dtype, device=x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size, dtype=x.dtype, device=x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

# Training loop with validation
def train_and_validate(model, criterion, optimizer, train_loader, val_loader, epochs, device, patience=20, min_delta=0.001):
    history = {'train_loss': [], 'val_loss': []}

    best_val_loss = float('inf')
    epochs_no_improve = 0
    
    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        total_iterations = len(train_loader)
        epoch_start_time = time.time()
        for sequences, labels, _, _ in train_loader:  
            sequences, labels = sequences.to(device), labels.to(device)
            labels = labels.unsqueeze(1) 
            optimizer.zero_grad()
            outputs = model(sequences)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        
        epoch_end_time = time.time()
        epoch_time = epoch_end_time - epoch_start_time
        train_loss /= len(train_loader)
        history['train_loss'].append(train_loss)

        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for sequences, labels, _, _ in val_loader:  
                sequences, labels = sequences.to(device), labels.to(device)
                labels = labels.unsqueeze(1)  
                outputs = model(sequences)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

        val_loss /= len(val_loader)
        history['val_loss'].append(val_loss)

        # Early stopping logic
        if val_loss < best_val_loss - min_delta:
            best_val_loss = val_loss
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1

        print(f'Epoch {epoch + 1}/{epochs}, Train Loss: {train_loss}, Validation Loss: {val_loss}')
        print(f'Time taken for epoch: {epoch_time:.8f} seconds')

        if epochs_no_improve >= patience:
            print('Early stopping triggered')
            # break

    return history

temperatures_to_process = ['25degC', '0degC', 'n10degC', 'n20degC', '10degC', '40degC']   
# temperatures_to_process = ['25degC', '0degC', '10degC'] 
data = load_data(PROCESSED_DATA_DIR, temperatures_to_process)
# Apply a moving average filter to your features
#window_size = 5 
#smoothed_data = data[FEATURE_COLS].rolling(window=window_size, min_periods=1).mean()
# Replace the original features with the smoothed features
#data[FEATURE_COLS] = smoothed_data

scaler = StandardScaler()
data[FEATURE_COLS] = scaler.fit_transform(data[FEATURE_COLS])

unique_files = np.array(list(set(data['SourceFile'])))

# Convert to tensors and load into GPU memory
data_tensor = torch.tensor(data[FEATURE_COLS].values, dtype=torch.float32).to(device)

labels_tensor = torch.tensor(data[LABEL_COL].values, dtype=torch.float32).to(device)

filenames_tensor = data['SourceFile'].values
unique_filelabels_tensors = np.array(list(set(data['SourceFile'])))
train_files, temp_files = train_test_split(unique_files, test_size=0.2, random_state=24)
val_files, test_files = train_test_split(temp_files, test_size=0.5, random_state=24)

def filter_data_by_filenames(df, filenames):
    return df[df['SourceFile'].isin(filenames)]

# Filter data for each set
train_data = filter_data_by_filenames(data, train_files)
val_data = filter_data_by_filenames(data, val_files)
test_data = filter_data_by_filenames(data, test_files)

# Convert to tensors
train_tensor = torch.tensor(train_data[FEATURE_COLS].values, dtype=torch.float32).to(device)
train_labels = torch.tensor(train_data[LABEL_COL].values, dtype=torch.float32).to(device)

val_tensor = torch.tensor(val_data[FEATURE_COLS].values, dtype=torch.float32).to(device)
val_labels = torch.tensor(val_data[LABEL_COL].values, dtype=torch.float32).to(device)

test_tensor = torch.tensor(test_data[FEATURE_COLS].values, dtype=torch.float32).to(device)
test_labels = torch.tensor(test_data[LABEL_COL].values, dtype=torch.float32).to(device)


# Convert filtered data to tensors and create dataset instances
train_dataset = BatteryDatasetLSTM(
    torch.tensor(train_data[FEATURE_COLS].values, dtype=torch.float32).to(device),
    torch.tensor(train_data[LABEL_COL].values, dtype=torch.float32).to(device),
    SEQUENCE_LENGTH,
    train_data['SourceFile'].values,
    train_data['Time [s]'].values  
)

val_dataset = BatteryDatasetLSTM(
    torch.tensor(val_data[FEATURE_COLS].values, dtype=torch.float32).to(device),
    torch.tensor(val_data[LABEL_COL].values, dtype=torch.float32).to(device),
    SEQUENCE_LENGTH,
    val_data['SourceFile'].values,
    val_data['Time [s]'].values  
)

test_dataset = BatteryDatasetLSTM(
    torch.tensor(test_data[FEATURE_COLS].values, dtype=torch.float32).to(device),
    torch.tensor(test_data[LABEL_COL].values, dtype=torch.float32).to(device),
    SEQUENCE_LENGTH,
    test_data['SourceFile'].values,
    test_data['Time [s]'].values  
)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)  
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Print file names used in training, validation, and testing
train_files = train_dataset.get_unique_filenames()
val_files = val_dataset.get_unique_filenames()
test_files = test_dataset.get_unique_filenames()

train_files_sorted = sorted(train_files)
val_files_sorted = sorted(val_files)
test_files_sorted = sorted(test_files)

print("Training files:", train_files)
print("\nValidation files:", val_files)
print("\nTesting files:", test_files)

Training files: {'562_Mixed4_processed.csv', '571_Mixed4_processed.csv', '551_LA92_processed.csv', '556_US06_processed.csv', '567_Mixed1_processed.csv', '576_UDDS_processed.csv', '551_UDDS_processed.csv', '604_Mixed3_processed.csv', '611_Mixed8_processed.csv', '596_LA92_processed.csv', '590_Mixed4_processed.csv', '551_US06_processed.csv', '571_Mixed5_processed.csv', '556_Mixed2_processed.csv', '589_LA92_processed.csv', '604_Mixed6_processed.csv', '562_Mixed7_processed.csv', '590_PausCycl_processed.csv', '557_Cap_1C_processed.csv', '562_Mixed8_processed.csv', '610_Cap_1C_processed.csv', '552_Cap_1C_processed.csv', '589_Mixed2_processed.csv', '589_Cap_1C_processed.csv', '551_Mixed2_processed.csv', '590_Mixed5_processed.csv', '571_Mixed7_processed.csv', '571_Mixed6_processed.csv', '590_Mixed6_processed.csv', '552_Mixed5_processed.csv', '552_Mixed8_processed.csv', '596_UDDS_processed.csv', '604_PausCycl_processed.csv', '610_Mixed1_processed.csv', '610_US06_processed.csv', '571_PausCycl_pro

In [15]:
print("Train features shape:", train_tensor.shape)
print("Test features shape:", test_tensor .shape)
print("Train labels shape:", train_labels.shape)
print("Test labels shape:", test_labels.shape)

Train features shape: torch.Size([420861, 5])
Test features shape: torch.Size([61263, 5])
Train labels shape: torch.Size([420861])
Test labels shape: torch.Size([61263])


In [ ]:
if not skip_optuna:
    # Define Optuna objective function
    def objective(trial):
        # Suggest hyperparameters
        hidden_size = trial.suggest_int('hidden_size', 10, 100)
        num_layers = trial.suggest_int('num_layers', 1, 5)
        learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-1, log=True)

        # Instantiate model with suggested hyperparameters
        model = SoCLSTM(input_size=len(FEATURE_COLS), hidden_size=hidden_size, num_layers=num_layers).type(torch.float32).to(device)

        # Define your loss function and optimizer with suggested hyperparameters
        criterion = nn.MSELoss()
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)

        # Call your train and validate function
        history = train_and_validate(model, criterion, optimizer, train_loader, val_loader, EPOCHS, device)

        # Extract last validation loss
        last_val_loss = history['val_loss'][-1]
        return last_val_loss

    # Optuna study
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=10)

    # Extract best trial
    best_trial = study.best_trial
    print(f"Best trial: {best_trial}")

    best_hyperparams = study.best_trial.params
    print('Best hyperparameters:', best_hyperparams)

    # Visualize the optimization process
    from optuna.visualization import plot_optimization_history, plot_param_importances

    # Plot optimization history
    optimization_history = plot_optimization_history(study)
    optimization_history.show()

    # Plot hyperparameter importance
    param_importances = plot_param_importances(study)
    param_importances.show()

[I 2025-08-16 11:28:36,232] A new study created in memory with name: no-name-53bbf3de-35d5-4d88-bb4b-d41990ebff96


Epoch 1/10, Train Loss: 0.002181748628898801, Validation Loss: 2.6241700185838425e-05
Time taken for epoch: 38.83314562 seconds
Epoch 2/10, Train Loss: 3.209049079507628e-05, Validation Loss: 1.5855683356745332e-05
Time taken for epoch: 39.20676970 seconds
Epoch 3/10, Train Loss: 3.24181607232845e-05, Validation Loss: 3.953341003261986e-05
Time taken for epoch: 40.20965314 seconds
Epoch 4/10, Train Loss: 2.4503042915724633e-05, Validation Loss: 2.740366933625603e-05
Time taken for epoch: 39.53385305 seconds
Epoch 5/10, Train Loss: 1.994172907857131e-05, Validation Loss: 1.3666707649790853e-05
Time taken for epoch: 38.85701156 seconds
Epoch 6/10, Train Loss: 1.7953212227356524e-05, Validation Loss: 1.51868439945107e-05
Time taken for epoch: 38.96214557 seconds
Epoch 7/10, Train Loss: 1.854761634661609e-05, Validation Loss: 1.0251330749293836e-05
Time taken for epoch: 38.68102241 seconds
Epoch 8/10, Train Loss: 1.4211206433233832e-05, Validation Loss: 1.8074191558077235e-05
Time taken fo

[I 2025-08-16 11:35:52,794] Trial 0 finished with value: 1.1132040123602718e-05 and parameters: {'hidden_size': 67, 'num_layers': 4, 'learning_rate': 0.01536023076194241}. Best is trial 0 with value: 1.1132040123602718e-05.


Epoch 10/10, Train Loss: 1.2615237243259054e-05, Validation Loss: 1.1132040123602718e-05
Time taken for epoch: 38.91143632 seconds
Epoch 1/10, Train Loss: 0.0009125531309153351, Validation Loss: 3.1682444231466803e-05
Time taken for epoch: 38.12938643 seconds
Epoch 2/10, Train Loss: 1.7131584069565403e-05, Validation Loss: 1.8451092882626588e-05
Time taken for epoch: 37.89919257 seconds
Epoch 3/10, Train Loss: 1.4299947950535712e-05, Validation Loss: 1.069612173667341e-05
Time taken for epoch: 37.62581134 seconds
Epoch 4/10, Train Loss: 1.254947393235941e-05, Validation Loss: 1.0748170470990832e-05
Time taken for epoch: 38.07316422 seconds
Epoch 5/10, Train Loss: 1.1212607275984536e-05, Validation Loss: 2.1065343589694496e-05
Time taken for epoch: 37.26248693 seconds
Epoch 6/10, Train Loss: 1.0823811832019457e-05, Validation Loss: 1.0759796463207952e-05
Time taken for epoch: 37.37732553 seconds
Epoch 7/10, Train Loss: 9.886450583103718e-06, Validation Loss: 1.2935505092857195e-05
Time 

[I 2025-08-16 11:42:50,685] Trial 1 finished with value: 1.1440828566427236e-05 and parameters: {'hidden_size': 75, 'num_layers': 2, 'learning_rate': 0.0008345306374400352}. Best is trial 0 with value: 1.1132040123602718e-05.


...

[I 2025-08-16 12:36:36,227] Trial 8 finished with value: 1.6388179656298742e-05 and parameters: {'hidden_size': 70, 'num_layers': 4, 'learning_rate': 0.0004959890764784705}. Best is trial 2 with value: 1.0647839426713764e-05.


Epoch 10/10, Train Loss: 8.940196619043022e-06, Validation Loss: 1.6388179656298742e-05
Time taken for epoch: 42.86067533 seconds
Epoch 1/10, Train Loss: 0.006911267238481779, Validation Loss: 0.00018398703681066334
Time taken for epoch: 39.88009024 seconds
Epoch 2/10, Train Loss: 7.950386316915993e-05, Validation Loss: 3.65337739580605e-05
Time taken for epoch: 39.49731207 seconds
Epoch 3/10, Train Loss: 2.0107142632893143e-05, Validation Loss: 1.4983307292541912e-05
Time taken for epoch: 40.35275769 seconds
Epoch 4/10, Train Loss: 1.2978576465693306e-05, Validation Loss: 1.015501322054746e-05
Time taken for epoch: 39.88385057 seconds
Epoch 5/10, Train Loss: 1.1431902614587534e-05, Validation Loss: 1.0951690330994722e-05
Time taken for epoch: 39.63033462 seconds
Epoch 6/10, Train Loss: 1.07706826891803e-05, Validation Loss: 1.096681490218882e-05
Time taken for epoch: 41.33282137 seconds
Epoch 7/10, Train Loss: 1.0238696151525253e-05, Validation Loss: 1.1672315627257877e-05
Time taken 

[I 2025-08-16 12:44:02,462] Trial 9 finished with value: 1.3743975382628708e-05 and parameters: {'hidden_size': 32, 'num_layers': 3, 'learning_rate': 0.0002534383221159352}. Best is trial 2 with value: 1.0647839426713764e-05.


Epoch 10/10, Train Loss: 9.104310056912402e-06, Validation Loss: 1.3743975382628708e-05
Time taken for epoch: 43.06514525 seconds
Best trial: FrozenTrial(number=2, state=1, values=[1.0647839426713764e-05], datetime_start=datetime.datetime(2025, 8, 16, 11, 42, 50, 685294), datetime_complete=datetime.datetime(2025, 8, 16, 11, 50, 17, 930520), params={'hidden_size': 41, 'num_layers': 5, 'learning_rate': 8.747203416460033e-05}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'hidden_size': IntDistribution(high=100, log=False, low=10, step=1), 'num_layers': IntDistribution(high=5, log=False, low=1, step=1), 'learning_rate': FloatDistribution(high=0.1, log=True, low=1e-05, step=None)}, trial_id=2, value=None)
Best hyperparameters: {'hidden_size': 41, 'num_layers': 5, 'learning_rate': 8.747203416460033e-05}


ImportError: Tried to import 'plotly' but failed. Please make sure that the package is installed correctly to use this feature. Actual error: No module named 'plotly'.

In [2]:
    # Optuna study
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=10)

    # Extract best trial
best_trial = study.best_trial
print(f"Best trial: {best_trial}")

best_hyperparams = study.best_trial.params
print('Best hyperparameters:', best_hyperparams)

from optuna.visualization import plot_optimization_history, plot_param_importances

# Plot optimization history
optimization_history = plot_optimization_history(study)
optimization_history.show()

# Plot hyperparameter importance
param_importances = plot_param_importances(study)
param_importances.show()

NameError: name 'optuna' is not defined

In [5]:
import os
skip_training = os.getenv('SKIP_TRAINING', 'False') == 'True'
if not skip_training:
    # Initialize the model, loss function, and optimizer
    best_hyperparams = {'hidden_size': 94, 'num_layers': 4, 'learning_rate': 3.5218624376812995e-05}
    hidden_size = best_hyperparams['hidden_size']
    num_layers = best_hyperparams['num_layers']
    
    EPOCHS = 20
    LEARNING_RATE = best_hyperparams['learning_rate']

    model = SoCLSTM(input_size=len(FEATURE_COLS), hidden_size=hidden_size, num_layers=num_layers)
    model.to(device).type(torch.float32)
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
    criterion = nn.MSELoss()

    # Train and validate the model
    history = train_and_validate(model, criterion, optimizer, train_loader, val_loader, EPOCHS, device)

    # Plot training history
    plt.figure(figsize=(10, 5))
    plt.plot(history['train_loss'], label='Train Loss')
    plt.plot(history['val_loss'], label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss Over Epochs')
    plt.legend()
    plt.grid(True)
    plt.show()

    # Save the model
    model_path = "soc_lstm_model.pth"
    torch.save({'model_state_dict': model.state_dict(), 'input_size': len(FEATURE_COLS)}, model_path)

NameError: name 'SoCLSTM' is not defined